In [1]:
from datasets import load_dataset, Dataset
from tqdm import tqdm
import openai
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

/home/tkdrnjs0621/miniconda3/envs/qwen2a2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [3]:
dialogue = load_dataset("12kimih/HiCUPID","dialogue")

In [4]:
formatted = {}

for item in tqdm(dialogue['train']):
    user_id = item['user_id']
    dialogue_id = item['dialogue_id']
    
    # Initialize nested structure if not exists
    if user_id not in formatted:
        formatted[user_id] = {}
    if dialogue_id not in formatted[user_id]:
        formatted[user_id][dialogue_id] = []
    
    # Append user and assistant turns as a list
    formatted[user_id][dialogue_id].append(item['user'])
    formatted[user_id][dialogue_id].append(item['assistant'])

100%|██████████| 331250/331250 [00:24<00:00, 13766.49it/s]


In [5]:
formatted_str = {}
for uid, dialogs in formatted.items():
    formatted_str[uid] = {}
    for did, turns in dialogs.items():
        lines = []
        for i, text in enumerate(turns):
            prefix = 'User:' if i % 2 == 0 else 'Assistant:'
            lines.append(f'{prefix} {text}')
        formatted_str[uid][did] = "\n".join(lines)

In [6]:
print(formatted_str[0][0])

User: What are the best ways to maintain rainboots for long-term use?
Assistant: To ensure your rainboots last for years, clean them regularly with mild soap and water, avoiding harsh chemicals. Store them in a cool, dry place away from direct sunlight to prevent material degradation. Additionally, applying a rubber conditioner can help maintain their flexibility and appearance.
User: I usually wear my rainboots daily. Would this affect their longevity?
Assistant: Wearing rainboots daily can lead to increased wear and tear, but proper care can mitigate this. Rotate between pairs if possible to allow them to rest and recover. Inspect them frequently for signs of damage and address issues promptly.
User: I find them very comfortable for my outdoor activities. Are there specific insoles recommended for rainboots?
Assistant: Yes, using insoles designed for rainboots can enhance comfort and support. Look for moisture-wicking and cushioned insoles to keep your feet dry and comfortable. Custo

In [ ]:

# completion = client.chat.completions.create(
#     model="gpt-4o-2024-08-06",
#     messages= [
#     {"role": "system", "content": """You will be given a conversation between the user and the assistant.
# Your task is to create **two questions and answers**, and the reference text that each question is based on, based on this conversation :

# 1. A **recall question** that asks about something said in the dialogue.
# 2. A **personalization question** that requests a recommendation based on the dialogue.

# The output format must be:

# ```
# RR: {reference text for recall question answering}
# RQ: {recall question}  
# RA: {answer for recall question}
# PR: {reference text for personalization question answering}
# PQ: {personalization question}  
# PA: {answer for personalization question}  
# ```

# * The **recall question** should be phrased like: *"What did I/you say about …"* and must directly reference something from the original conversation.
# * The **personalization question** should be phrased like: *"Recommend some …"* and should be clearly reference the user's preference or experience from the original question.
# * Both questions must be strongly tied to the given dialogue so that, even after many later conversations, asking either question would naturally make the assistant recall this specific dialogue.
# * Both questions must be answer properly **only if** the given conversation is used to answer the question.
# * Make sure the recall question sounds as natural as possible—something the user might genuinely forget and later ask the assistant about after some time.
# * Make sure the personalization question naturally asks questions that requires user's perference, and the user's preference must not be explicitly mentioned in the question.
# * The question is asked by the user, and the answer is answered by assistant, so use the proper pronoun for question and answers.
# * Reference text must be a exact copy of a line from the conversation, starting with "User:", or "Assistant:".
# * The reference text must be the reference for each answers, if the answer is based on the user's question, it should be user's utterance. If it's based on the assistant's answer, it must be assistant's utterance.
# """.strip()},
#     {"role": "user", "content": formatted_str[0][0]},
# ],
#     temperature=1,
#     top_p=1,
# )

# print(completion.choices[0].message.content)

```
RR: User: I live in a region with frequent rain. Are there any accessories that complement rainboots?
RQ: What did you ask about accessories that complement rainboots for frequent rainy conditions?  
RA: You asked if there are any accessories that complement rainboots, and I mentioned boot liners, waterproof sprays, and boot bags as useful additions to enhance your rainboot experience.

PR: User: I enjoy exploring nature trails. Are there any specific features to look for in rainboots for such activities?
PQ: Can you recommend some rainboots with specific features for exploring nature trails?  
PA: For exploring nature trails, I recommend rainboots with sturdy soles for traction, reinforced toes for protection, lightweight designs for comfort, and waterproof and breathable materials for extended wear in varying conditions.  
```


In [7]:

completion = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages= [
    {"role": "system", "content": """You will be given a conversation between the user and the assistant.
You will be given a conversation between the user and the assistant.
Your task is to create a questions and answers based on this conversation, speicifically, a **recall question** that asks about something said in the dialogue.
The output format must be:

```
Q: {recall question}  
A: {answer for recall question}
```

* The **recall question** should be phrased like: *"What did I/you say about …"* and must directly reference something from the original conversation.
* The question must be strongly tied to the given dialogue so that, even after many later conversations, asking the question would naturally make the assistant recall this specific dialogue.
* The question must be answer properly **only if** the given conversation is used to answer the question.
* Make sure the question sounds as natural as possible—something the user might genuinely forget and later ask the assistant about after some time.
* The question is asked by the user, and the answer is answered by assistant, so use the proper pronoun for question and answer.
""".strip()},
    {"role": "user", "content": formatted_str[0][0]},
],
    temperature=1,
    top_p=1,
)

print(completion.choices[0].message.content)

```
Q: What did you say about maintaining rainboots for long-term use?  
A: To ensure your rainboots last for years, clean them regularly with mild soap and water, avoiding harsh chemicals. Store them in a cool, dry place away from direct sunlight to prevent material degradation. Additionally, applying a rubber conditioner can help maintain their flexibility and appearance.
```
